# Implementación de K-nearest-neighbors

* Oskar Adolfo Villa
* Cruz Daniel Pérez
* Rogelio Hernández

## Imports

In [4]:
import pandas as pd
import math
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score


## Implementación

La siguiente clase contiene todo el código necesario para ejecutar un algoritmo de KNN con distancia euclidiana.

In [ ]:
import math

class KNearestNeighbors:
    def __init__(self, k=3):
        self.k = k
        self.data = []

    def fit(self, X, y):
        self.data = [(x, label) for x, label in zip(X, y)]

    def predict(self, X_test):
        predictions = []
        results = []

        for idx, x in enumerate(X_test):
            neighbors = self._get_neighbors(x)
            # assigned_class = self._vote(neighbors)
            assigned_class = self._vote_manual(neighbors)
            class_count = self._count_neighbors(neighbors)

            results.append({
                "Instance": idx + 1,
                "tested_negative": class_count.get("tested_negative", 0),
                "tested_positive": class_count.get("tested_positive", 0),
                "Assigned class": assigned_class
            })

            predictions.append(assigned_class)

        return predictions, results

    def _get_neighbors(self, x):
        distances = []
        for point, label in self.data:
            dist = self._euclidean_distance(x, point)
            distances.append((dist, label))
        distances.sort(key=lambda d: d[0])
        return distances[:self.k]
    
    # def _vote(self, neighbors):
    #     votes = {}
    #     for _, label in neighbors:
    #         votes[label] = votes.get(label, 0) + 1
    #     return max(votes, key=votes.get)

    @staticmethod
    def _euclidean_distance(x1, x2):
        return math.sqrt(sum((a - b) ** 2 for a, b in zip(x1, x2)))

    def _count_neighbors(self, neighbors):
        count = {}
        for _, label in neighbors:
            if label in count:
                count[label] += 1
            else:
                count[label] = 1
        return count

    def _vote_manual(self, neighbors):
        count = self._count_neighbors(neighbors)
        assigned = None
        max_votes = -1
        for label, num in count.items():
            if num > max_votes:
                max_votes = num
                assigned = label
        return assigned


## Ejecución

In [8]:
def run_knn(k=3, normalize=False):
    train = pd.read_csv("Diabetes-Entrenamiento.csv")
    test = pd.read_csv("Diabetes-Clasificacion.csv")

    X_train = train.drop(columns=["class"]).values
    y_train = train["class"].values
    X_test = test.drop(columns=["class"]).values
    y_test = test["class"].values

    if normalize:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

    knn = KNearestNeighbors(k)
    knn.fit(X_train, y_train)
    predictions, results = knn.predict(X_test)

    results_df = pd.DataFrame(results)
    results_df["True class"] = y_test
    results_df["Correct"] = results_df["Assigned class"] == results_df["True class"]
    
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy with k = {k}, normalized = {normalize}: {accuracy * 100:.2f}%")

    return results_df


### Ejecución con distintos valores de k y normalización

In [10]:
res1 = run_knn(k=1, normalize=False)
res2 = run_knn(k=3, normalize=False)
res3 = run_knn(k=7, normalize=False)

res4 = run_knn(k=1, normalize=True)
res5 = run_knn(k=3, normalize=True)
res6 = run_knn(k=7, normalize=True)


Accuracy with k = 1, normalized = False: 67.00%
Accuracy with k = 3, normalized = False: 70.00%
Accuracy with k = 7, normalized = False: 74.00%
Accuracy with k = 1, normalized = True: 72.00%
Accuracy with k = 3, normalized = True: 83.00%
Accuracy with k = 7, normalized = True: 77.00%


## Conclusiones


1. **¿Cómo varía el resultado con diferentes valores de k?**  
    Se observó que, sin normalización, la precisión del algoritmo KNN fue aumentando ligeramente al incrementar el valor de k: con k=1 se obtuvo un 67.00%, con k=3 un 70.00%, y con k=7 un 74.00%. Esto indica que al considerar más vecinos, el modelo logra tomar decisiones más estables, lo que mejora la precisión. Sin embargo, en el caso con datos normalizados, el mejor resultado se obtuvo con k=3 (83.00%), mientras que k=1 y k=7 dieron precisiones menores (72.00% y 77.00%, respectivamente), lo que sugiere que hay un punto óptimo para k y no siempre un valor mayor es mejor.
2. **¿Tiene inferencia en el resultado el que los datos estén normalizados?**  
    Sí, la normalización tuvo un impacto significativo en los resultados. Comparando cada valor de k, se obtuvo una mayor precisión cuando los datos estaban normalizados. Por ejemplo, con k=3, la precisión pasó de 70.00% sin normalizar a 83.00% con normalización. Esto confirma que al usar distancia euclidiana, la normalización es esencial para evitar que atributos con valores grandes dominen el cálculo de la distancia.

3. **¿Cuáles fueron los resultados mínimos y máximos obtenidos en los experimentos?**  
    El resultado mínimo fue de 67.00% (k=1 sin normalización), y el máximo fue de 83.00% (k=3 con normalización). Esto representa una mejora del 16% entre el peor y el mejor escenario, lo cual demuestra la importancia tanto del valor de k como de la normalización en el rendimiento del algoritmo KNN.


### Guardar resultados en archivo CSV y mostrar precisión

In [11]:
def save_and_display(results_df, file_name):
    results_df[["Instance", "tested_negative", "tested_positive", "Assigned class"]].to_csv(file_name, index=False)
    correct = results_df["Correct"].sum()
    total = len(results_df)
    accuracy = 100 * correct / total
    print(f"Archivo guardado como {file_name}")
    print(f"Precisión: {accuracy:.2f}% ({correct}/{total})")

save_and_display(res2, "resultados_k3.csv")
save_and_display(res5, "resultados_k3_normalizado.csv")


Archivo guardado como resultados_k3.csv
Precisión: 70.00% (70/100)
Archivo guardado como resultados_k3_normalizado.csv
Precisión: 83.00% (83/100)
